### The following scripts used for extracting comments from youtube videos using Youtube API.

In [ ]:
# THE WALL STREET JOURNAL
# 29 Jan 2025 (362 DAYS) in 27 Jan 2026
# Video title: How Trump Is Planning to End Birthright Citizenship | WSJ

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "xIkLgooN9LA"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"1-29-25_WSJ_29_Jan_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")


Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 2979

First 5 comments:
  Comment ID           Author                 Time  \
0         C1             @wsj  2025-01-29T15:05:06   
1         C2      @inframatic  2025-01-29T15:54:57   
2         C3  @tomfullery9710  2025-01-29T15:57:51   
3         C4      @jasonj1376  2025-01-29T15:58:52   
4         C5       @Sam-yf4kt  2025-01-29T16:05:36   

                                        Comment Text  Likes  Reply Count  
0  Trump sows fear among migrants with shock-and-...    169           52  
1  Legal immigrants have nothing to fear, and all...     53            0  
2  WSJ sows fear by misrepresenting the truth.  F...      1            0  
3  Illegals should be in fear, as any wh break th...     27            0  
4          Illegals - that was a convenient omission     22            0  

All comments saved to: YouTube_Comments\1-29-25_WSJ_29_Jan_2025_xIkLgooN9LA.xlsx


In [ ]:
# CNN
# 6 Dec 2025
# Video title: Supreme Court agrees to decide if Trump may end birthright citizenship

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "1H-sg4h43DY"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"12-6-25_CNN_6_Dec_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 4404

First 5 comments:
  Comment ID             Author                 Time  \
0         C1   @matiaslanzi6132  2026-01-25T09:27:28   
1         C2      @iTrapForKash  2026-01-17T15:09:53   
2         C3       @TeamSpinUSA  2026-01-05T07:22:46   
3         C4  @ohlsenjulian3288  2026-01-03T06:48:43   
4         C5      @tonyotto4930  2025-12-29T21:32:45   

                                        Comment Text  Likes  Reply Count  
0  Hahaha, this is not possible. Are you crazy? I...      0            0  
1                           No more anchor ⚓️ baby’s      0            0  
2  If approved by the Supreme Court for ending bi...      0            0  
3  14th amendment was for black slaves not refuge...      0            0  
4  It's not about Donald Trump being able to chan...      7            1  

All comments saved to: YouTube_Comments\12-6-25_CNN_6_Dec_2025_1H-sg4h43DY.xlsx


In [ ]:
# New York Times
# 15 May 2025
# Video title: Can Trump Really End Birthright Citizenship?

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "gYg1Jaz6G3I"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"5-15-25_NYT_15_May_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 894

First 5 comments:
  Comment ID            Author                 Time  \
0         C1       @CMyChannel  2026-01-26T14:55:24   
1         C2        @WulfStrom  2026-01-17T23:39:03   
2         C3          @EazyE69  2026-01-01T13:01:48   
3         C4    @LucasPolk-j5n  2025-12-31T05:28:58   
4         C5  @ramakrishna5480  2025-12-27T08:40:34   

                                        Comment Text  Likes  Reply Count  
0                             No more anchor babies.      0            0  
1  So that will open the crack for potentially be...      0            0  
2           Laws are overturned all the time, enjoy.      0            0  
3  Something must be done about this. You can't i...      0            0  
4                                 1898 case really 😂      0            0  

All comments saved to: YouTube_Comments\5-15-25_NYT_15_May_2025_gYg1Jaz6G3I.xlsx


In [ ]:
# ABC News
# 5 Dec 2025
# Video title: Supreme Court to take up President Trump's birthright citizenship case

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "XNpA19NMnHU"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"12-5-25_ABC_5_Dec_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 1637

First 5 comments:
  Comment ID            Author                 Time  \
0         C1      @JudahEl-w2q  2026-01-27T02:26:11   
1         C2  @da-profezor5178  2026-01-26T04:55:32   
2         C3   @rubinyoung9355  2026-01-13T22:01:54   
3         C4    @lesasveen9982  2025-12-31T07:02:39   
4         C5  @TanyaPepper-p6x  2025-12-31T06:55:36   

                                        Comment Text  Likes  Reply Count  
0  Ok how is this valid? That will be only the in...      0            0  
1               We need to start with his kids first      0            0  
2  December 12, 2025\n\nDear President Donald J. ...      0            0  
3  Native Americans were excluded from birthright...      3            0  
4                                          Go Trump!      2            0  

All comments saved to: YouTube_Comments\12-5-25_ABC_5_Dec_2025_XNpA19NMnHU.xlsx


In [ ]:
# NBC News
# 9 Dec 2024
# Video title: Trump says he’d end birthright citizenship through executive action if he can

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "2LnMTjpuO1o"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"12-9-24_NBC_9_Dec_2024_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 2455

First 5 comments:
  Comment ID                  Author                 Time  \
0         C1  @Alexandria.Washington  2026-01-05T00:26:19   
1         C2        @NajahSalman-z1u  2025-05-18T16:03:02   
2         C3     @LorenzoNambo-jt7yp  2025-05-16T15:16:32   
3         C4            @southendbos  2025-05-09T14:28:10   
4         C5      @conniejohnson9511  2025-05-09T02:59:37   

                                        Comment Text  Likes  Reply Count  
0  Autochthon Indigenous “Black” \nAmerican-India...      0            0  
1  This man is very bad or crazy he wants to ruin...      0            0  
2   That will lower the migration to the USA by 100%      0            0  
3  Except for Rubio. Rubio is a birthright citize...      0            0  
4  We’re not the only country that has it.  He ab...      0            0  

All comments saved to: YouTube_Comments\12-9-24_NBC_

In [ ]:
# Democracy Now!
# 19 Dec 2025
# Video title: Trump Moves to Denaturalize Citizens, End Birthright Citizenship, Halt Visa Lottery

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "W8dXD8-z6-4"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"12-19-25_DemoNow_19_Dec_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 6655

First 5 comments:
  Comment ID                    Author                 Time  \
0         C1           @joyceyaffe1832  2026-01-26T23:56:59   
1         C2               @laurab9867  2026-01-26T23:06:52   
2         C3           @alinalindt4676  2026-01-26T21:50:35   
3         C4        @jorgemelendez9546  2026-01-26T03:37:21   
4         C5  @lynneleeluckdowsing6654  2026-01-26T01:01:09   

                                        Comment Text  Likes  Reply Count  
0  Trump's wife was born in another country. Shou...      1            0  
1  If you are an immigrant... \n\nPROTECT YOURSEL...      0            0  
2  Seems like war criminals are welcomed as long ...      0            0  
3  He’s the president to gobern that’s why was el...      1            0  
4  All countries should inform the US that they a...      0            0  

All comments saved to: YouTube_Comments\

In [ ]:
# CNN
# 29 Sept 2025
# Video title: Trump asks Supreme Court to end birthright citizenship

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "A8CY5W9Ek0I"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"9-29-25_CNN_29_Sept_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 1186

First 5 comments:
  Comment ID            Author                 Time  \
0         C1  @abbyonomorelies  2025-12-13T19:47:04   
1         C2    @USsuperiority  2025-10-28T16:01:44   
2         C3  @NWstockoperator  2025-11-11T04:56:57   
3         C4  @paulaharvey1171  2025-11-13T13:14:33   
4         C5  @NWstockoperator  2025-11-13T15:45:34   

                                        Comment Text  Likes  Reply Count  
0  14th Amendment written to benefit freed slaves...      1            0  
1  So what will happen to Barron Trump son of an ...      1            3  
2  Baron and Don jrs father is a US citizen. Wher...      1            0  
3  Legal immigrant. Nothing to do with illegal fo...      0            0  
4                               ​@paulaharvey1171yes      1            0  

All comments saved to: YouTube_Comments\9-29-25_CNN_29_Sept_2025_A8CY5W9Ek0I.xlsx


In [ ]:
# KMPH FOX26 NEWS
# 6 Dec 2025
# Video title: A case to end birthright citizenship

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "pZlO5yvWVf4"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"12-6-25_KMPH_6_Dec_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 4058

First 5 comments:
  Comment ID            Author                 Time  \
0         C1  @greghammett5739  2026-01-19T00:30:06   
1         C2        @TrueMohax  2026-01-25T09:59:06   
2         C3  @greghammett5739  2026-01-25T14:12:05   
3         C4        @TrueMohax  2026-01-25T18:43:41   
4         C5  @greghammett5739  2026-01-25T20:08:52   

                                        Comment Text  Likes  Reply Count  
0  Stop the criminal actions of the Supreme Goons...      0            4  
1                No one is changing the Constitution      0            0  
2  ​@TrueMohaxAnyone else want to respond to this...      0            0  
3  @greghammett5739\nWhat is he doing with those ...      0            0  
4  ​@TrueMohaxFirst and foremost anyone that tell...      0            0  

All comments saved to: YouTube_Comments\12-6-25_KMPH_6_Dec_2025_pZlO5yvWVf4.xlsx


In [ ]:
# CBS NEWS
# 25 Jul 2025
# Video title: Trump's birthright citizenship order faces legal blow

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "cecRYTlseIg"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"7-25-25_CBS_25_Jul_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 638

First 5 comments:
  Comment ID           Author                 Time  \
0         C1  @marksteele3876  2026-01-02T08:29:32   
1         C2       @sheepbay3  2025-12-24T13:57:48   
2         C3    @beepuffs5871  2025-12-29T10:48:40   
3         C4         @pandy7x  2025-12-08T03:28:00   
4         C5     @johndye9463  2025-11-26T13:15:04   

                                        Comment Text  Likes  Reply Count  
0  Ironically unless you’re a NATIVE, no-one in A...      0            0  
1  I hope Trump wins This !!!!!!    We need to pr...      0            1  
2  Come on how does it affect anybody in any way ...      0            0  
3  Natives are probably like " F'YEAH. "  What's ...      0            0  
4  Everyone knows that if it were anyone else the...      0            0  

All comments saved to: YouTube_Comments\7-25-25_CBS_25_Jul_2025_cecRYTlseIg.xlsx


In [ ]:
# FACE THE NATION
# 25 Jul 2025
# Video title: Trump's efforts to end birthright citizenship hit roadblock

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "KVn3H-biBhc"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"7-25-25_FACEZNATION_25_Jul_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 848

First 5 comments:
  Comment ID          Author                 Time  \
0         C1  @blackbird-one  2025-08-28T19:53:49   
1         C2     @koneos6580  2025-07-27T18:58:50   
2         C3         @zz532a  2025-07-27T14:34:11   
3         C4  @RoxannOkamura  2025-07-27T03:51:04   
4         C5  @gregkotyk3445  2025-07-27T03:21:03   

                                        Comment Text  Likes  Reply Count  
0  The framers of the 14th Amendment drew a clear...      1            0  
1  Oh mine. If the law passes, 99% of all America...      0            0  
2   Without birthright citizenship who is a citizen?      0            0  
3  What is his problem that's been a bill since t...      0            0  
4            Wake up America you are all immigrants.      0            1  

All comments saved to: YouTube_Comments\7-25-25_FACEZNATION_25_Jul_2025_KVn3H-biBhc.xlsx


In [ ]:
# CNN
# 10 Jul 2025
# Video title: Federal judge blocks Trump's birthright citizenship order

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "TpeeA3brCj4"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"7-10-25_CNN_10_Jul_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 405

First 5 comments:
  Comment ID                  Author                 Time  \
0         C1  @carolinerothstein5258  2025-10-11T22:37:51   
1         C2   @annabellesanchez8011  2025-08-06T17:33:06   
2         C3        @danielogrady303  2025-08-05T00:53:50   
3         C4        @skepticynic5150  2025-07-18T12:46:56   
4         C5                @uap1234  2025-07-15T19:15:43   

                                        Comment Text  Likes  Reply Count  
0  So now they want to protect the babies, then s...      0            0  
1      A federal judge has no say so! So ridiculous!      1            0  
2                             Yoyr not blocking shit      0            0  
3  Fine. The babies can stay. The illegal parents...      0            0  
4  All white americans could fit in one state. Wh...      1            0  

All comments saved to: YouTube_Comments\7-10-25_CNN_1

In [ ]:
# LiveNOW from FOX
# 27 Jun 2025
# Video title: "Meant for babies of slaves" Trump on birthright citizenship ruling

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "qgrhpN7K-Q4"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"6-27-25_LiveNOW_27_Jun_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 3816

First 5 comments:
  Comment ID                          Author                 Time  \
0         C1                   @ANDREW-eq8lf  2026-01-20T23:07:13   
1         C2                 @LOTSAPOPPA4765  2026-01-20T18:39:30   
2         C3  @joseoswaldomartinezrodrigu154  2026-01-17T09:32:28   
3         C4       @Cookie_monster_deathStar  2026-01-12T22:40:06   
4         C5                @lisawilliams345  2026-01-09T17:06:34   

                                        Comment Text  Likes  Reply Count  
0                                      Thank you 😊 ❤      0            0  
1  What all you immigrants are missing or ignorin...      0            0  
2         Te cruzó con una bestia y nazca una plaga.      0            0  
3                       God bless Donald J Trump 👍🇺🇲      0            0  
4  Seems like anything that was created to give r...      0            1  

All 

In [ ]:
# MS NOW
# 23 Jan 2025
# Video title: Trump attempts to end birthright citizenship

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "fA3KIogjDn4"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"1-23-25_MSNOW_23_Jan_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")


Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 1360

First 5 comments:
  Comment ID                           Author                 Time  \
0         C1           @Alexandria.Washington  2026-01-20T04:22:29   
1         C2                      @joetube144  2025-12-21T15:49:11   
2         C3  @squidgameisaliarisawnosqui5341  2025-12-16T13:47:34   
3         C4                      @julesj5853  2025-12-13T01:12:30   
4         C5                 @lauraalston8012  2025-11-27T23:07:39   

                                        Comment Text  Likes  Reply Count  
0  American SLAVES were - Imported “Africans”, wh...      0            0  
1  Not true. Read the Constitution word for word....      0            0  
2                                       Get them out      0            0  
3  That is not what the 14th ammendment says thou...      1            0  
4  Illegals have no right in our country!\nNo one...      0            0  

In [ ]:
# LiveNOW from FOX
# 27 Sept 2025
# Video title: BREAKING: Trump urges Supreme Court to allow birthright citizenship restrictions

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "VPYrw7XreYQ"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"9-27-25_LiveNOW_27_Sept_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 1339

First 5 comments:
  Comment ID             Author                 Time  \
0         C1           @pandy7x  2025-12-08T03:38:52   
1         C2        @njmcatnaps  2025-12-06T21:15:56   
2         C3      @Kathleen1212  2025-12-01T09:27:11   
3         C4      @Kathleen1212  2025-12-01T09:25:56   
4         C5  @susanehamlin8550  2025-11-30T15:55:15   

                                        Comment Text  Likes  Reply Count  
0  Natives are probably like " F'YEAH. "  What's ...      0            0  
1              BLOCK THEM, THANK YOU PRESIDENT TRUMP      0            0  
2                Exactly, OUR VOICE IS OUR VOTE!!!!!      0            0  
3  Yes Congress is failing us.  I voted for Presi...      0            0  
4  Donald J. Trump is also a birthright citizen, ...      0            0  

All comments saved to: YouTube_Comments\9-27-25_LiveNOW_27_Sept_2025_VPYrw7XreYQ.x

In [ ]:
# Valuetainment
# 11 Dec 2024
# Video title: Trump Vows to End Birthright Citizenship and Overhaul Costly Migrant Court System

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "7eRSz5GUA8A"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"12-11-24_Valuetainment_11_Dec_2024_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 6224

First 5 comments:
  Comment ID                Author                 Time  \
0         C1      @matthewware8973  2025-07-10T20:42:40   
1         C2          @Deleted8790  2025-05-10T07:39:04   
2         C3  @egyptiansoldier6535  2025-04-25T10:54:48   
3         C4      @matthewware8973  2025-07-10T20:43:11   
4         C5  @egyptiansoldier6535  2025-07-10T22:30:44   

                                        Comment Text  Likes  Reply Count  
0                                    Trump ❤ Epstein      0            0  
1               See how his vows turn out to be lies      0            0  
2  It is true tho. America is the only country th...      0            2  
3  Most north and south American countries have b...      0            0  
4  @matthewware8973right but you gotta agree tho ...      0            0  

All comments saved to: YouTube_Comments\12-11-24_Valuetainment_1

In [ ]:
# MS NOW
# 27 Jan 2026
# Video title: 'Unconstitutional': MN Republican ends bid for governor citing Trump's immigration policy

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "ow2aidHU44o"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"1-27-26_MSNOW_27_Jan_2026_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 1960

First 5 comments:
  Comment ID              Author                 Time  \
0         C1   @kennethwells2465  2026-01-27T16:06:19   
1         C2  @judyfernandez3100  2026-01-27T16:05:01   
2         C3            @mfsebcw  2026-01-27T16:02:56   
3         C4  @pinkawearnezz3231  2026-01-27T16:02:28   
4         C5       @YosefSchmidt  2026-01-27T16:02:25   

                                        Comment Text  Likes  Reply Count  
0  What about driving while black? Blacks still a...      0            0  
1  None of these attacks on U.S citizens is norma...      0            0  
2  What a lying sack of schitt. Nobody with an ou...      0            0  
3  Glad to see some of them actually have there o...      0            0  
4  If you want to find the mother load of murdere...      0            0  

All comments saved to: YouTube_Comments\1-27-26_MSNOW_27_Jan_2026_ow2aidHU44

In [ ]:
# First Post
# 18 Dec 2025
# Video title: Trump Immigration Crackdown to Hit Naturalised US Citizens In 2026 | Spotlight | N18G

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "VbH4d8gZs9I"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"12-18-25_FirstPost_18_Dec_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 861

First 5 comments:
  Comment ID                  Author                 Time  \
0         C1      @kondeladunami7918  2026-01-18T23:22:47   
1         C2           @anesevol5758  2026-01-17T22:37:25   
2         C3  @epsteinfilesfunisland  2026-01-14T00:28:51   
3         C4                 @bmax43  2026-01-12T03:21:16   
4         C5      @maureenmccall4381  2026-01-11T23:58:09   

                                        Comment Text  Likes  Reply Count  
0              President is prejudice against Latino      0            0  
1  "Honest mistakes". Why is it then that the sam...      0            0  
2  wow, white people REALLY want to keep America ...      0            0  
3  American citizens do not have to cowl down to ...      0            0  
4                                Start with mellania      0            0  

All comments saved to: YouTube_Comments\12-18-25_Firs

In [ ]:
# Forbes Breaking News
# 15 Jan 2026
# Video title: Trump Calls To Revoke Citizenship Of Any Naturalized Immigrant Convicted Of Defrauding Citizens

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "TVvl9CrU9XA"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"1-15-26_ForbesBreakingNews_15_Jan_2026_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 1465

First 5 comments:
  Comment ID               Author                 Time  \
0         C1    @therealcoachross  2026-01-26T06:10:45   
1         C2  @nicholecollins6126  2026-01-24T15:39:52   
2         C3   @altheatrotman5606  2026-01-23T23:44:53   
3         C4  @scarletfrances5004  2026-01-23T20:25:32   
4         C5            @lolately  2026-01-23T18:50:00   

                                        Comment Text  Likes  Reply Count  
0  Thank you President Trump! The American People...      0            0  
1  I wonder if this includes people like Peter Th...      0            0  
2                       That's right President Trump      0            0  
3  Please have the IRS take a look at their taxes...      0            0  
4  First off, why do we pay for childcare when th...      0            0  

All comments saved to: YouTube_Comments\1-15-26_ForbesBreakingNews_15_

In [ ]:
# CBS Evening News & CBS News
# 29 Nov 2025
# Video title: Trump says he plans to end immigration from "Third-World Countries"

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "4AyMeUNpnPU"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"11-29-25_CBSEVENING&CBS_29_Nov_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 880

First 5 comments:
  Comment ID                Author                 Time  \
0         C1  @kristelynmiller3084  2026-01-25T21:43:21   
1         C2       @PacificCircle1  2025-12-30T02:36:36   
2         C3        @cynthia121266  2025-12-28T09:25:41   
3         C4        @cynthia121266  2025-12-27T00:59:00   
4         C5        @cynthia121266  2025-12-27T00:55:22   

                                        Comment Text  Likes  Reply Count  
0  “Undermine domestic tranquility”! So, what abo...      0            0  
1     Going full racist and Vance is doing the same.      0            0  
2  🇺🇲🙁 in the state of  Nevada  apartments were s...      0            0  
3                   🇺🇲🙁 making the situations worse.      0            0  
4  🇺🇲🙁 many black or Africans  don't have  where ...      0            0  

All comments saved to: YouTube_Comments\11-29-25_CBSEVENING&CBS_2

In [ ]:
# CNN
# 26 Sept 2025
# Video title: Trump admin cracks down on legal immigration

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "8iaXBtTIq7w"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"9-26-25_CNN_26_Sept_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 2284

First 5 comments:
  Comment ID            Author                 Time  \
0         C1      @Mypurplehat  2026-01-22T21:53:45   
1         C2    @BushidoXBrown  2026-01-20T12:00:01   
2         C3    @jessepaul2180  2026-01-19T18:35:10   
3         C4  @carlamcneal1963  2026-01-19T18:29:57   
4         C5  @carlamcneal1963  2026-01-19T18:23:26   

                                        Comment Text  Likes  Reply Count  
0                     What about Melania and her son      0            0  
1  nazi\n\nthis is literally conservative extremi...      0            0  
2  nobody cares about your opinion illegal immigr...      0            0  
3                                       Thank God. 😂      0            0  
4  I think I love President Trump for getting unw...      0            0  

All comments saved to: YouTube_Comments\9-26-25_CNN_26_Sept_2025_8iaXBtTIq7w.xlsx


In [ ]:
# Darian Immigration Law
# 24 Nov 2024
# Video title: Trump's New Naturalization Policies Explained: What Immigrants Need to Know (so far)

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "JuyA3o2CTwI"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"11-24-24_DarianImmigrationLaw_24_Nov_2024_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 866

First 5 comments:
  Comment ID                Author                 Time  \
0         C1         @AndreasW-p4r  2025-04-02T13:42:48   
1         C2      @ayerimlopez5982  2025-04-02T02:53:34   
2         C3           @G.I.Sity17  2025-03-25T21:33:05   
3         C4       @bhammoudeh2794  2025-03-18T14:27:25   
4         C5  @robertsontirado4478  2025-03-14T23:21:49   

                                        Comment Text  Likes  Reply Count  
0  What about Melania trump and kids? I tell you ...      0            0  
1  I remember back when we became US Citizens it ...      0            0  
2                                               Come      0            0  
3       Does that affect old I-10 old applications ?      0            0  
4  A lot of non American women came to my face an...      0            0  

All comments saved to: YouTube_Comments\11-24-24_DarianImmigratio

In [ ]:
# NBC News
# 28 Nov 2025
# Video title: Trump administration orders reexamination of some green card holders

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "9fjF0lQihbc"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"11-28-25_NBC_28_Nov_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 204

First 5 comments:
  Comment ID                 Author                 Time  \
0         C1             @AmanSS890  2025-12-19T13:20:31   
1         C2         @infernus42254  2025-12-19T07:33:55   
2         C3  @aaronthebarbaric7733  2025-12-05T14:27:13   
3         C4         @MyThought2026  2025-12-03T10:45:08   
4         C5          @chefstirston  2025-12-04T11:23:05   

                                        Comment Text  Likes  Reply Count  
0  Canada needs to start  doing the same to many ...      0            0  
1                                             Peace.      0            0  
2  Trump Administration giving the heathens The P...      0            0  
3  The Judges to be blame for the death of the in...      3            1  
4                                          its trump      1            0  

All comments saved to: YouTube_Comments\11-28-25_NBC_28_Nov

In [ ]:
# MS NOW
# 7 Jul 2025
# Video title: 'Everybody is now on the chopping block': Trump DOJ now targeting naturalized citizens

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "MlXcO6_aA2Q"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"7-7-25_MSNOW_7_Jul_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 2875

First 5 comments:
  Comment ID               Author                 Time  \
0         C1        @agilbert8262  2026-01-13T07:32:00   
1         C2         @Auntietoshi  2026-01-04T17:43:14   
2         C3  @elenaroukhadze4518  2026-01-01T09:15:08   
3         C4    @cllowboyking5152  2025-12-31T17:14:25   
4         C5         @Neonfluxvid  2026-01-02T20:58:12   

                                        Comment Text  Likes  Reply Count  
0  And he is only a 3rd generation U.S. citizen. ...      0            0  
1       THIS BOILS DOWN TO A SUPREME DICTATORSHIP!😂❤      0            0  
2  Bravo republicans! Simultaneously turning 46 m...      0            0  
3  If trump is going after naturalized citizens, ...      1            1  
4                                     Elon Musk too.      0            0  

All comments saved to: YouTube_Comments\7-7-25_MSNOW_7_Jul_2025_MlXcO6

In [ ]:
# CBS NEWS
# 7 Nov 2025
# Video title: Trump administration may deny visas for immigrants with certain health conditions

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "NAE_zdHROP8"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"11-7-25_CBS_7_Nov_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 391

First 5 comments:
  Comment ID               Author                 Time  \
0         C1  @colleenjackson4227  2026-01-15T03:56:13   
1         C2  @DieselTechie-uo9we  2026-01-03T18:37:36   
2         C3             @Snowlys  2025-12-26T02:26:31   
3         C4          @AbLL-zf3ev  2025-12-02T15:12:25   
4         C5       @anEvilRat2009  2025-11-28T21:13:14   

                                        Comment Text  Likes  Reply Count  
0                                         Way to go!      0            0  
1  Is that a joke?  We are on the opposite side o...      0            0  
2  This is the least we could do. Should be doing...      0            0  
3  Americans restrict visa to most countries but ...      0            0  
4  Step 1: Ruin their country for sake of oil and...      0            0  

All comments saved to: YouTube_Comments\11-7-25_CBS_7_Nov_2025_NAE_zdHR

In [ ]:
# NBC NEWS
# 21 Jan 2025
# Video title: Trump birthright citizenship executive order expected to face legal challenges

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "BhFXB9lYcIE"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"1-21-25_NBC_21_Jan_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 864

First 5 comments:
  Comment ID             Author                 Time  \
0         C1         @DianaNuhn  2025-07-30T16:21:09   
1         C2  @jonahjohnsen6519  2025-05-11T02:28:54   
2         C3  @jonahjohnsen6519  2025-05-11T02:25:37   
3         C4            @jf4872  2025-05-04T07:18:23   
4         C5       @sunnymittra  2025-03-14T19:56:28   

                                        Comment Text  Likes  Reply Count  
0  Birthright citizenship is not for illegals fro...      1            0  
1  What Trump appears to be doing.\n\nHypothetica...      0            0  
2  The Quiet Revolution Against Birthright Citize...      0            0  
3  The recent SCOTUS ruling made clear....Being b...      0            0  
4  People deliberately give birth to children in ...      0            0  

All comments saved to: YouTube_Comments\1-21-25_NBC_21_Jan_2025_BhFXB9lYcIE.xlsx


In [ ]:
# Washington Post Universe
# 28 Mar 2025
# Video title: ‘Proof of citizenship’ executive order explained

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "g6OPnGaWawA"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"3-28-25_WashingtonPostUniverse_28_Mar_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 2904

First 5 comments:
  Comment ID          Author                 Time  \
0         C1  @DaveJorgenson  2025-03-27T23:10:47   
1         C2     @bowler7922  2025-03-27T23:16:06   
2         C3   @asthetic8131  2025-03-27T23:18:36   
3         C4      @伏見猿比古-k8c  2025-03-27T23:48:31   
4         C5   @THEDUDE32967  2025-03-28T00:26:20   

                                        Comment Text  Likes  Reply Count  
0                   We have Schoolhouse Rock at home   7261           26  
1                                                  😂     21            0  
2               Bill would be rolling in his grave 💔     51            0  
3        I don't want the schoolhouse rock at home 😭     17            0  
4  I'm just a Bill, a possibly Unlawful Bill, Kic...     45            0  

All comments saved to: YouTube_Comments\3-28-25_WashingtonPostUniverse_28_Mar_2025_g6OPnGaWawA.xlsx


In [ ]:
# Darian Immigration Law
# 30 Nov 2024
# Video title: Trump's New Marriage Green Card Policies Explained: What Spouses Need to Know (so far)

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "dHAwN4MognI"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"11-30-24_DarianImmigrationLaw_30_Nov_2024_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 1429

First 5 comments:
  Comment ID                     Author                 Time  \
0         C1         @kaywasinjapan9497  2025-07-16T17:24:09   
1         C2         @CREATIVEJUNKI8050  2025-05-24T05:01:24   
2         C3  @trdrockwarrior_sport4432  2025-05-16T06:38:19   
3         C4  @trdrockwarrior_sport4432  2025-05-16T06:37:05   
4         C5  @trdrockwarrior_sport4432  2025-05-16T06:36:16   

                                        Comment Text  Likes  Reply Count  
0  I wish they made it illegal to marry American ...      0            0  
1  Brother every “risk” you mentioned it’s not up...      0            0  
2  What is this they can optioned themselves they...      0            0  
3  This 90 day bull crap needs to be changed to 3...      0            0  
4  This 90 day bull crap needs to be changed to 3...      0            0  

All comments saved to: YouTube_Com

In [ ]:
# ABC NEWS
# 27 Jun 2025
# Video title: FULL SPECIAL REPORT: Trump speaks following SCOTUS ruling on birthright citizenship order

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "oOsC8W04Asw"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"6-27-25_ABC_27_Jun_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 881

First 5 comments:
  Comment ID              Author                 Time  \
0         C1   @martinbernal3161  2025-09-28T02:16:20   
1         C2     @kittycat-tn3rr  2025-09-24T02:39:07   
2         C3     @kittycat-tn3rr  2025-09-24T02:38:54   
3         C4  @FlaccoZLuvsMariah  2025-07-24T01:44:23   
4         C5       @MdRashad-i4v  2025-07-19T05:28:52   

                                        Comment Text  Likes  Reply Count  
0  I really hope that the US citizenship can be t...      0            0  
1                                              11:41      1            0  
2                                              11:40      0            0  
3                                   !!A KHLOE!!🗯️🐒⚠️      0            0  
4  Md,nurul,Islam,md,asab,Ali,sonamukhi,kazipur,s...      1            0  

All comments saved to: YouTube_Comments\6-27-25_ABC_27_Jun_2025_oOsC8W04Asw.x

In [ ]:
# Top 10 Tales
# 2 Apr 2025
# Video title: The END of DUAL Citizenship? Trump's SHOCKING New Plan Revealed!

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "X5qjDDEBraw"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"4-2-25_Top10Tales_2_Apr_2025_{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 600

First 5 comments:
  Comment ID            Author                 Time  \
0         C1       @sculpting2  2025-12-04T12:10:16   
1         C2      @PyroShields  2025-12-02T19:11:31   
2         C3       @sliferxxxx  2025-12-02T15:54:23   
3         C4      @PyroShields  2025-12-02T19:12:59   
4         C5  @royalalpha.1377  2025-12-02T02:11:18   

                                        Comment Text  Likes  Reply Count  
0  Absolutely against constitution and freedom.  ...      0            0  
1  Does his new plan includes his wife or is this...      0            0  
2  Hmm watch them have exceptions for politicians...      0            1  
3                                   Or Trump's wife.      0            0  
4                               Run it on them Trump      0            0  

All comments saved to: YouTube_Comments\4-2-25_Top10Tales_2_Apr_2025_X5qjDDEBraw.xlsx


In [ ]:
# MS NOW
# 8 Dec 2025 
# Video title: Growing reports of U.S. citizens being detained amid Trump's immigration crackdown

import os
import time
from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime
import re

# ----------------- CONFIG -----------------
api_key = " "  # type your YouTube API key here
video_id = "3wknU22QVZI"  # YouTube video ID
max_results_per_page = 100  # Max comments per API call (max 100)
output_folder = "YouTube_Comments"  # Folder to save Excel files
delay_between_requests = 0.1  # seconds
max_retries = 3  # retries if a page fails
# -----------------------------------------

def clean_text_for_excel(text):
    """Remove characters that can break Excel."""
    if isinstance(text, str):
        return re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f]", "", text)
    return text

def fetch_comments(youtube, video_id):
    """Fetch top-level comments and all replies."""
    comments_data = []
    comment_id_counter = 1
    next_page_token = None

    while True:
        retries = 0
        while retries < max_retries:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=max_results_per_page,
                    pageToken=next_page_token,
                    textFormat="plainText"
                )
                response = request.execute()
                break
            except Exception as e:
                retries += 1
                print(f"Error fetching page {next_page_token}, retry {retries}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Failed to fetch page {next_page_token}, skipping...")
            break

        for item in response.get('items', []):
            top_snippet = item['snippet']['topLevelComment']['snippet']
            comment_id = f"C{comment_id_counter}"
            comments_data.append({
                'Comment ID': comment_id,
                'Author': top_snippet['authorDisplayName'],
                'Time': datetime.strptime(top_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                'Comment Text': clean_text_for_excel(top_snippet['textDisplay']),
                'Likes': top_snippet['likeCount'],
                'Reply Count': item['snippet'].get('totalReplyCount', 0)
            })
            comment_id_counter += 1

            # If top-level comment has replies, fetch all
            total_replies = item['snippet'].get('totalReplyCount', 0)
            if total_replies > 0:
                parent_id = item['id']
                reply_next_token = None
                while True:
                    retries_reply = 0
                    while retries_reply < max_retries:
                        try:
                            reply_request = youtube.comments().list(
                                part="snippet",
                                parentId=parent_id,
                                maxResults=max_results_per_page,
                                pageToken=reply_next_token,
                                textFormat="plainText"
                            )
                            reply_response = reply_request.execute()
                            break
                        except Exception as e:
                            retries_reply += 1
                            print(f"Error fetching replies for {parent_id}, retry {retries_reply}/{max_retries}: {e}")
                            time.sleep(1)
                    else:
                        print(f"Failed to fetch replies for {parent_id}, skipping...")
                        break

                    for reply in reply_response.get('items', []):
                        reply_snippet = reply['snippet']
                        comment_id = f"C{comment_id_counter}"
                        comments_data.append({
                            'Comment ID': comment_id,
                            'Author': reply_snippet['authorDisplayName'],
                            'Time': datetime.strptime(reply_snippet['publishedAt'], "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%S"),
                            'Comment Text': clean_text_for_excel(reply_snippet['textDisplay']),
                            'Likes': reply_snippet['likeCount'],
                            'Reply Count': 0  # replies to replies are not fetched further
                        })
                        comment_id_counter += 1

                    reply_next_token = reply_response.get('nextPageToken')
                    if not reply_next_token:
                        break
                    time.sleep(delay_between_requests)

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(delay_between_requests)

    return comments_data

# ----------------- MAIN SCRIPT -----------------
if __name__ == "__main__":
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    youtube = build('youtube', 'v3', developerKey=api_key)

    print("Scraping comments and all replies, this may take some time...")
    comments = fetch_comments(youtube, video_id)
    print(f"Total comments fetched (including replies): {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Display first 5 rows
        print("\nFirst 5 comments:")
        print(df.head())

        # Save to Excel
        excel_filename = os.path.join(output_folder, f"12-8-25_MSNOW_8_Dec_2025 _{video_id}.xlsx")
        df.to_excel(excel_filename, index=False)
        print(f"\nAll comments saved to: {excel_filename}")
    else:
        print("No comments found.")

Scraping comments and all replies, this may take some time...
Total comments fetched (including replies): 870

First 5 comments:
  Comment ID                Author                 Time  \
0         C1          @kristof_123  2025-12-31T13:22:50   
1         C2            @JayProper  2025-12-23T14:49:16   
2         C3  @lawandacontreras767  2025-12-14T15:34:59   
3         C4      @WeirdScience-ET  2025-12-14T02:49:26   
4         C5    @michaelthomas1916  2025-12-13T20:30:34   

                                        Comment Text  Likes  Reply Count  
0  We heard Dar and Cathy Yarbrough were reported...      1            0  
1  Now they get to experience what being black in...      0            0  
2  It won’t stop until Congress gets off their A*...      0            0  
3  We have not heard the story behind why these w...      0            0  
4  Reasonable suspension is them not being the wh...      0            0  

All comments saved to: YouTube_Comments\12-8-25_MSNOW_8_Dec_2025 

,channel,video_date,video_title
0,THE WALL STREET JOURNAL,2025-01-29,How Trump Is Planning to End Birthright Citize...
1,CNN,2025-12-06,Supreme Court agrees to decide if Trump may en...
2,New York Times,2025-05-15,Can Trump Really End Birthright Citizenship?
3,ABC News,2025-12-05,Supreme Court to take up President Trump's bir...
4,NBC News,2024-12-09,Trump says he’d end birthright citizenship thr...
5,Democracy Now!,2025-12-19,"Trump Moves to Denaturalize Citizens, End Birt..."
6,29 Sept 2025,2025-12-19,Trump asks Supreme Court to end birthright cit...
7,KMPH FOX26 NEWS,2025-12-06,A case to end birthright citizenship
8,CBS NEWS,2025-07-25,Trump's birthright citizenship order faces leg...
9,FACE THE NATION,2025-07-25,Trump's efforts to end birthright citizenship ...


,channel,video_date,video_id,video_title
0,THE WALL STREET JOURNAL,2025-01-29,None,How Trump Is Planning to End Birthright Citize...
1,CNN,2025-12-06,None,Supreme Court agrees to decide if Trump may en...
2,New York Times,2025-05-15,None,Can Trump Really End Birthright Citizenship?
3,ABC News,2025-12-05,None,Supreme Court to take up President Trump's bir...
4,NBC News,2024-12-09,None,Trump says he’d end birthright citizenship thr...
5,Democracy Now!,2025-12-19,None,"Trump Moves to Denaturalize Citizens, End Birt..."
6,29 Sept 2025,2025-12-19,None,Trump asks Supreme Court to end birthright cit...
7,KMPH FOX26 NEWS,2025-12-06,None,A case to end birthright citizenship
8,CBS NEWS,2025-07-25,None,Trump's birthright citizenship order faces leg...
9,FACE THE NATION,2025-07-25,None,Trump's efforts to end birthright citizenship ...
